In [12]:
import csv
import gzip
from dateutil.parser import parse
from decimal import *
import pandas as pd
import gc
import os
from multiprocessing import Process
def calculate_longevity(name):
    info(name)
    cnt = 0
    snapshots = {}
    longevity = {}
    snapshot_id = 0
    for lines in pd.read_csv("/mnt/17volume/data/snapshot_revision_git.csv.gz.part" + name, chunksize=1000, encoding='utf-8', header=None):
        for line in lines.iterrows():
            #print(line[1])
            if cnt == 0 and name == 'aa':
                print("first process aa")
                cnt+=1
                continue
            try:    
                if line[1][2] == line[1][2]:
                    if line[1][0] not in snapshots:
                        if len(snapshots) > 0:
                            snapshots[snapshot_id].sort()
                            #print(len(snapshots[snapshot_id]))
                            if len(snapshots[snapshot_id]) > 1:
                                m = abs(int(snapshots[snapshot_id][-1]) - int(snapshots[snapshot_id][0]) )

                                #print(snapshot_id,' ', longevity[snapshot_id])
                                s  = pd.Series(longevity,index=longevity.keys())
                                df = pd.DataFrame({
                                    'snapshot_id':[snapshot_id], 
                                    'longevity':[m]
                                })
                                #print(df)
                                df.to_csv('/home/sv/longevity-' + name + '.csv.gz', compression = 'gzip', mode ='a', header=False, index=False)
                            del snapshots[snapshot_id]
                        del snapshots
                        gc.collect()
                        snapshots = {}
                        snapshot_id = line[1][0]
                        if snapshot_id is not None:
                            snapshots[snapshot_id] = [line[1][2]]
                    else:
                        snapshots[snapshot_id].append(line[1][2])
            except Exception as e:
                f = open("/home/sv/longevity_error.txt", "a")
                f.write("Project id: " + str(line[1][0]))
                f.write(", error: " + str(e) + "\n")
                with open('/home/sv/longevity_exception.csv', mode='a') as project_file:
                    project_writer = csv.writer(project_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                    project_writer.writerow([line[1][0], line[1][2], line[1][3], line[1][1]])
                pass
def info(title):
    print(title)
    print('module name:', __name__)
    print('parent process:', os.getppid())
    print('process id:', os.getpid())
    
if __name__ == '__main__':
    p1 = Process(target=calculate_longevity, args = ('aa',))
    p1.start()
    p2 = Process(target=calculate_longevity, args = ('ab',))
    p2.start()
    p3 = Process(target=calculate_longevity, args = ('ac',))
    p3.start()  
    p4 = Process(target=calculate_longevity, args = ('ad',))
    p4.start()  
    p5 = Process(target=calculate_longevity, args = ('ae',))
    p5.start()
    p6 = Process(target=calculate_longevity, args = ('af',))
    p6.start()

aa
('module name:', '__main__')
('parent process:', 8746)
ab
('process id:', 6692)
ac
('module name:', '__main__')
('parent process:', 8746)
('module name:', '__main__')
('process id:', 6697)
ad
('module name:', '__main__')
('parent process:', 8746)
('parent process:', 8746)
('process id:', 6706)
('process id:', 6713)
ae
('module name:', '__main__')
('parent process:', 8746)
('process id:', 6724)
af
('module name:', '__main__')
('parent process:', 8746)
('process id:', 6737)
first process aa


Process Process-63:
Process Process-64:
Process Process-65:
Process Process-66:
Traceback (most recent call last):
Traceback (most recent call last):
Process Process-62:
Traceback (most recent call last):
  File "/usr/lib/python2.7/multiprocessing/process.py", line 267, in _bootstrap
    self.run()
  File "/usr/lib/python2.7/multiprocessing/process.py", line 114, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-12-ae3bf3d9fbff>", line 23, in calculate_cycle_times
    if line[1][2] == line[1][2]:
  File "/home/sv/.local/lib/python2.7/site-packages/pandas/core/series.py", line 868, in __getitem__
  File "/usr/lib/python2.7/multiprocessing/process.py", line 267, in _bootstrap
    result = self.index.get_value(self, key)
    self.run()
  File "/usr/lib/python2.7/multiprocessing/process.py", line 114, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-12-ae3bf3d9fbff>", line 16, in calculate_cycle_times
  File "/home/sv/.local/lib/python2.